In [14]:
from libsvm.svmutil import *
from liblinear.liblinearutil import *
from sklearn.metrics import accuracy_score
import multiprocessing as mp
import numpy as np
import itertools
import random
import math
from math import log, exp
import sys

In [15]:
# Load data
y_train, X_train = svm_read_problem("letter_train")
y_test, X_test = svm_read_problem("letter_test")


In [16]:
# Label the training data
X_tr = []
y_tr = []

for num in range(len(y_train)):
    if y_train[num] == 11:
        new = 1
        y_tr.append(new)
        X_tr.append([X_train[num][k] for k in range(1, len(X_train[num]) + 1)])
    elif y_train[num] == 26:
        new = -1
        y_tr.append(new)
        X_tr.append([X_train[num][k] for k in range(1, len(X_train[num]) + 1)])

# Label the test data
X_t = []
y_t = []

for num in range(len(y_test)):
    if y_test[num] == 11:
        new = 1
        y_t.append(new)
        X_t.append([X_test[num][k] for k in range(1, len(X_test[num]) + 1)])
    elif y_test[num] == 26:
        new = -1
        y_t.append(new)
        X_t.append([X_test[num][k] for k in range(1, len(X_test[num]) + 1)])


In [17]:
def get_error_rate(pred, true, weights):
    # Calculate the error rate using the weighted 0/1 error
    error = sum(w for i, w in enumerate(weights) if pred[i] != true[i])
    return error / sum(weights)

def decision_stump(X, y, weights):
    # Implement the decision stump algorithm
    num_samples, num_features = len(X), len(X[0])
    best_error = float('inf')
    best_feature, best_threshold, best_s = None, None, None

    for i in range(num_features):
        sorted_indices = sorted(range(num_samples), key=lambda x: X[x][i])
        thresholds = [float('-inf')] + [(X[sorted_indices[j]][i] + X[sorted_indices[j+1]][i]) / 2 for j in range(num_samples - 1)]
        
        for threshold in thresholds:
            for s in [-1, 1]:
                pred = [s if x[i] < threshold else -s for x in X]
                error = get_error_rate(pred, y, weights)

                if error < best_error:
                    best_error = error
                    best_feature = i
                    best_threshold = threshold
                    best_s = s
    
    return best_feature, best_threshold, best_s, best_error

def adaboost_stump(X, y, T):
    num_samples = len(X)
    num_features = len(X[0])

    # Initialize weights
    weights = [1 / num_samples] * num_samples

    alphas = []
    classifiers = []

    for t in range(T):
        # Train a decision stump
        feature, threshold, s, error = decision_stump(X, y, weights)

        # Calculate alpha
        alpha = 0.5 * log((1 - error) / error)
        alphas.append(alpha)

        # Update weights
        Z = sum(weights)
        weights = [w * exp(-alpha * p * t) / Z for w, p, t in zip(weights, [s if x[feature] < threshold else -s for x in X], y)]

        # Save the classifier
        classifiers.append((feature, threshold, s))
        # Print progress
        print("Iteration", t + 1, "completed")

    return alphas, classifiers


In [38]:
def stump_predict(x, stump):
    feature = stump[0]       # Access feature index using index 0
    threshold = stump[1]     # Access threshold using index 1
    s = stump[2]             # Access s value using index 2

    return s if x[feature] < threshold else -s

def evaluate(X, y, alphas, classifiers):
    num_samples = len(X)
    num_classifiers = len(classifiers)

    errors = []

    for i in range(num_samples):
        prediction = sum(alpha * stump_predict(X[i], classifier) for alpha, classifier in zip(alphas, classifiers))
        errors.append(int(np.sign(prediction) != y[i]))

    ein_G = sum(alpha * error for alpha, error in zip(alphas, errors)) / num_samples
    eout_G = sum(errors) / num_samples

    return ein_G, eout_G

In [ ]:
# Run AdaBoost-Stump on the training data
T = 1000
alphas, classifiers = adaboost_stump(X_tr, y_tr, T)


In [39]:
# Calculate Ein(G), Eout(G)
ein_G, _ = evaluate(X_tr, y_tr, alphas, classifiers)
_, eout_G = evaluate(X_t, y_t, alphas, classifiers)

# Calculate min_{1≤t≤1000} Ein(gt)
min_ein = min(get_error_rate([s if X_tr[i][feature] < threshold else -s for i in range(len(X_tr))], y_tr, [1] * len(X_tr)) for feature, threshold, s in classifiers)
# Calculate max_{1≤t≤1000} Ein(gt)
max_ein = max(get_error_rate([s if X_tr[i][feature] < threshold else -s for i in range(len(X_tr))], y_tr, [1] * len(X_tr)) for feature, threshold, s in classifiers)

# Print the results
print("min_{1≤t≤1000} Ein(gt):", min_ein)
print("max_{1≤t≤1000} Ein(gt):", max_ein)

print("Ein(G):", "{:.5f}".format(ein_G))
print("Eout(G):", "{:.5f}".format(eout_G))

min_{1≤t≤1000} Ein(gt): 0.09846547314578005
max_{1≤t≤1000} Ein(gt): 0.571611253196931
Ein(G): 0.00000
Eout(G): 0.00279
